## Prepare Dependencies

In [1]:
import pandas as pd

from os import listdir
from os.path import isfile, join

## Wrangle Data

In [2]:
file_names = []
for file in listdir('saga/english/'):
    file_names.append(str(file))

In [3]:
#Import the text from every document as an entry in a dataframe
corpus = {}
raw = pd.DataFrame()

for book in file_names:
    corpus[str(book)] = open('saga/english/{}'.format(str(book)))

## Clean Text